In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if int(x) == x else '{:,.2f}'.format(x)
pd.set_option('display.max_columns', 30)

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [2]:
def graph_pie(data, labels, title):
    fig = {
      "data": [
        {
          "values": data,
          "labels": labels,
          "domain": {"x": [0, .48]},
          "name": "",
          "hoverinfo":"label+value+percent",
          "hole": .4,
          "type": "pie",
          "sort": False
        }],
      "layout": {
            "title": title,
            "annotations": [
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": "",
                    "x": 0.20,
                    "y": 0.5
                }
            ]
        }
    }
    iplot(fig, filename=title)


In [3]:
contract_sigef = pd.ExcelFile("../Consolidado/Queries/Data Contrato SIGEF Junio 2018.xlsx").parse("Hoja1")
contract_sigef.MesAprobacion = contract_sigef.MesAprobacion.apply(lambda x: x.title()[:3])

In [4]:
MONTH = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]

## Compras mensuales

In [5]:
contracts_by_month = pd.pivot_table(contract_sigef, values=["IdentificacionDeContrato", "ValorTotal"], index=["MesAprobacion"], aggfunc={"IdentificacionDeContrato":np.count_nonzero, "ValorTotal":np.sum})
contracts_by_month = pd.DataFrame(contracts_by_month, index=MONTH).fillna(0)
contracts_by_month.columns = ["Cantidad", "Monto"]
contracts_by_month = contracts_by_month.transpose()
contracts_by_month

,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic
Cantidad,843,1215,1603,1520,1768,1415,0,0,0,0,0,0
Monto,"1,449,347,854.73","1,181,287,255.32","550,863,081.91","1,394,368,966.06","1,970,414,147.73","1,497,647,766.91",0,0,0,0,0,0


In [6]:
graph_pie(data=contracts_by_month.loc["Cantidad"], labels=contracts_by_month.columns, title="Cantidad de contratos por mes")

In [7]:
graph_pie(data=contracts_by_month.loc["Monto"], labels=contracts_by_month.columns, title="Monto contratados por mes")

## Compras mensuales por tipo de proveedor

In [8]:
CLASIF_RPE = ["MIPYMES no certificadas y otras organizaciones", "Gran empresa", "MIPYMES Certificadas por el MIC", "Persona Fisica"]
contract_sigef.ClasificacionRPE = contract_sigef.ClasificacionRPE.fillna("Persona Fisica")
contract_sigef.ClasificacionRPE = contract_sigef.ClasificacionRPE.replace("No Especificada", "MIPYMES no certificadas y otras organizaciones")
contract_sigef.ClasificacionRPE = contract_sigef.ClasificacionRPE.replace("No clasificada", "MIPYMES no certificadas y otras organizaciones")

contract_sigef.loc[(contract_sigef.CertificadoPorMIC=="Si") & (contract_sigef.ClasificacionRPE=="Pequeña empresa"), "ClasificacionRPE"] = "MIPYMES Certificadas por el MIC"
contract_sigef.loc[(contract_sigef.CertificadoPorMIC=="Si") & (contract_sigef.ClasificacionRPE=="Micro empresa"), "ClasificacionRPE"] = "MIPYMES Certificadas por el MIC"
contract_sigef.loc[(contract_sigef.CertificadoPorMIC=="Si") & (contract_sigef.ClasificacionRPE=="Mediana empresa"), "ClasificacionRPE"] = "MIPYMES Certificadas por el MIC"

contract_sigef.loc[(contract_sigef.CertificadoPorMIC=="No") & (contract_sigef.ClasificacionRPE=="Pequeña empresa"), "ClasificacionRPE"] = "MIPYMES no certificadas y otras organizaciones"
contract_sigef.loc[(contract_sigef.CertificadoPorMIC=="No") & (contract_sigef.ClasificacionRPE=="Micro empresa"), "ClasificacionRPE"] = "MIPYMES no certificadas y otras organizaciones"
contract_sigef.loc[(contract_sigef.CertificadoPorMIC=="No") & (contract_sigef.ClasificacionRPE=="Mediana empresa"), "ClasificacionRPE"] = "MIPYMES no certificadas y otras organizaciones"

In [9]:
contracts_by_month_prov = pd.pivot_table(contract_sigef, index=["MesAprobacion", "ClasificacionRPE"], values=["IdentificacionDeContrato", "ValorTotal"], aggfunc={"IdentificacionDeContrato":np.count_nonzero, "ValorTotal":np.sum})
contracts_by_month_prov = pd.DataFrame(contracts_by_month_prov, index=[(m, c) for m in MONTH for c in CLASIF_RPE]).fillna(0)
contracts_by_month_prov.columns = ["Cantidad", "Monto"]

dic = contracts_by_month_prov.to_dict()
new_dic = {m:{(t, c):dic[t][(m,c)] for c in CLASIF_RPE for t in ["Cantidad", "Monto"]} for m in MONTH}

contracts_by_month_prov = pd.DataFrame.from_dict(new_dic)

In [10]:
contracts_by_month_prov

Ene  \
Cantidad Gran empresa                                               39   
         MIPYMES Certificadas por el MIC                           255   
         MIPYMES no certificadas y otras organizaciones            502   
         Persona Fisica                                             47   
Monto    Gran empresa                                   537,359,373.01   
         MIPYMES Certificadas por el MIC                121,376,549.22   
         MIPYMES no certificadas y otras organizaciones 744,889,191.84   
         Persona Fisica                                  45,722,740.66   

                                                                   Feb  \
Cantidad Gran empresa                                               56   
         MIPYMES Certificadas por el MIC                           338   
         MIPYMES no certificadas y otras organizaciones            711   
         Persona Fisica                                            110   
Monto    Gran empresa                                    47,881,957.55   
         MIPYMES Certificadas por el MIC                222,842,694.17   
         MIPYMES no certificadas y otras organizaciones 888,177,739.14   
         Persona Fisica                                  22,384,864.46   

                                                                   Mar  \
Cantidad Gran empresa                                               88   
         MIPYMES Certificadas por el MIC                           457   
         MIPYMES no certificadas y otras organizaciones            937   
         Persona Fisica                                            121   
Monto    Gran empresa                                     8,868,125.62   
         MIPYMES Certificadas por el MIC                126,977,274.73   
         MIPYMES no certificadas y otras organizaciones 391,433,748.36   
         Persona Fisica                                  23,583,933.20   

                                                                   Abr  \
Cantidad Gran empresa                                               65   
         MIPYMES Certificadas por el MIC                           410   
         MIPYMES no certificadas y otras organizaciones            922   
         Persona Fisica                                            123   
Monto    Gran empresa                                    17,068,089.01   
         MIPYMES Certificadas por el MIC                307,981,476.34   
         MIPYMES no certificadas y otras organizaciones 976,068,659.93   
         Persona Fisica                                  93,250,740.78   

                                                                     May  \
Cantidad Gran empresa                                                 73   
         MIPYMES Certificadas por el MIC                             514   
         MIPYMES no certificadas y otras organizaciones             1067   
         Persona Fisica                                              114   
Monto    Gran empresa                                     300,856,888.52   
         MIPYMES Certificadas por el MIC                  267,767,062.09   
         MIPYMES no certificadas y otras organizaciones 1,388,772,423.05   
         Persona Fisica                                    13,017,774.07   

                                                                     Jun  Jul  \
Cantidad Gran empresa                                                 70    0   
         MIPYMES Certificadas por el MIC                             380    0   
         MIPYMES no certificadas y otras organizaciones              888    0   
         Persona Fisica                                               77    0   
Monto    Gran empresa                                      42,423,544.77    0   
         MIPYMES Certificadas por el MIC                  393,424,429.69    0   
         MIPYMES no certificadas y otras organizaciones 1,040,882,482.71    0   
         Persona Fisica                                    20,917,

In [11]:
data = [go.Bar(
        x = contracts_by_month_prov.columns,
        y = contracts_by_month_prov.loc[("Cantidad", clasif)],
        name = clasif
        ) for clasif in CLASIF_RPE]
iplot(go.Figure(data, layout=go.Layout(title="Cantidad de contratos por tipo de proveedor")))

In [12]:
data = [go.Bar(
        x = contracts_by_month_prov.columns,
        y = contracts_by_month_prov.loc[("Monto", clasif)],
        name = clasif
        ) for clasif in CLASIF_RPE]
iplot(go.Figure(data, layout=go.Layout(title="Monto contratados por tipo de proveedor")))

## Modalidad de compras por tipo de proveedor

In [13]:
MOD = ["Comparación de Precios o Competencia", "Compras por encima del umbral", "Compras Menores", "Compras por debajo del umbral mínimo", "Licitación Privada/Restringida", "Licitación pública", "Proceso de Excepción", "Sorteo de Obras"]
contract_sigef.ModalidadCompra = contract_sigef.ModalidadCompra.replace("Compra Directa", "Compras por encima del umbral")
contracts_by_prov_mod = pd.pivot_table(contract_sigef, index=["ModalidadCompra", "ClasificacionRPE"], values=["IdentificacionDeContrato", "ValorTotal"], aggfunc={"IdentificacionDeContrato":np.count_nonzero, "ValorTotal":np.sum})
contracts_by_prov_mod = pd.DataFrame(contracts_by_prov_mod, index=[(m, c) for m in MOD for c in CLASIF_RPE]).fillna(0)
contracts_by_prov_mod.columns = ["Cantidad", "Monto"]

dic = contracts_by_prov_mod.to_dict()
new_dic = {m:{(t, c):dic[t][(m,c)] for c in CLASIF_RPE for t in ["Cantidad", "Monto"]} for m in MOD}

contracts_by_prov_mod = pd.DataFrame.from_dict(new_dic)

In [14]:
contracts_by_prov_mod

Comparación de Precios o Competencia  \
Cantidad Gran empresa                                                                       3   
         MIPYMES Certificadas por el MIC                                                   90   
         MIPYMES no certificadas y otras organizaciones                                   130   
         Persona Fisica                                                                    15   
Monto    Gran empresa                                                            4,596,395.77   
         MIPYMES Certificadas por el MIC                                       186,141,847.66   
         MIPYMES no certificadas y otras organizaciones                        488,374,849.71   
         Persona Fisica                                                         57,422,892.88   

                                                         Compras por encima del umbral  \
Cantidad Gran empresa                                                                0   
         MIPYMES Certificadas por el MIC                                             4   
         MIPYMES no certificadas y otras organizaciones                              6   
         Persona Fisica                                                              0   
Monto    Gran empresa                                                                0   
         MIPYMES Certificadas por el MIC                                  5,801,122.62   
         MIPYMES no certificadas y otras organizaciones                   4,186,263.53   
         Persona Fisica                                                              0   

                                                         Compras Menores  \
Cantidad Gran empresa                                                 61   
         MIPYMES Certificadas por el MIC                             670   
         MIPYMES no certificadas y otras organizaciones             1222   
         Persona Fisica                                              241   
Monto    Gran empresa                                      18,192,909.75   
         MIPYMES Certificadas por el MIC                  185,522,863.40   
         MIPYMES no certificadas y otras organizaciones   363,383,689.97   
         Persona Fisica                                    48,533,210.76   

                                                         Compras por debajo del umbral mínimo  \
Cantidad Gran empresa                                                                     250   
         MIPYMES Certificadas por el MIC                                                 1148   
         MIPYMES no certificadas y otras organizaciones                                  2460   
         Persona Fisica                                                                   305   
Monto    Gran empresa                                                            7,828,540.68   
         MIPYMES Certificadas por el MIC                                        44,088,501.28   
         MIPYMES no certificadas y otras organizaciones                         91,486,338.89   
         Persona Fisica                                                         11,289,567.65   

                                                         Licitación Privada/Restringida  \
Cantidad Gran empresa                                                                 0   
         MIPYMES Certificadas por el MIC                                              0   
         MIPYMES no certificadas y otras organizaciones                               0   
         Persona Fisica                                                               0   
Monto    Gran empresa                                                                 0   
         MIPYMES Certificadas por el MIC                                              0   
         MIPYMES no certificadas y otras organizaciones                               0   
         Persona Fisica                                                              

In [15]:
aux = contracts_by_prov_mod.transpose()["Cantidad"]
data = [go.Bar(
        x = aux.columns,
        y = aux.loc[m],
        name = m
        ) for m in MOD]
iplot(go.Figure(data, layout=go.Layout(title="Modalidad de compra por tipo de proveedor - Cantidad de contratos")))

In [16]:
aux = contracts_by_prov_mod.transpose()["Monto"]
data = [go.Bar(
        x = aux.columns,
        y = aux.loc[m],
        name = m
        ) for m in MOD]
iplot(go.Figure(data, layout=go.Layout(title="Modalidad de compra por tipo de proveedor - Monto Contratado")))

## Compras mensuales por modalidad

In [17]:
contracts_by_month_mod = pd.pivot_table(contract_sigef, index=["MesAprobacion", "ModalidadCompra"], values=["IdentificacionDeContrato", "ValorTotal"], aggfunc={"IdentificacionDeContrato":np.count_nonzero, "ValorTotal":np.sum})
contracts_by_month_mod = pd.DataFrame(contracts_by_month_mod, index=[(m, c) for m in MONTH for c in MOD]).fillna(0)
contracts_by_month_mod.columns = ["Cantidad", "Monto"]

dic = contracts_by_month_mod.to_dict()
new_dic = {(m, i):{(t, c):0 if i=="%" else dic[t][(m,c)] for c in MOD for t in ["Cantidad", "Monto"]} for m in MONTH for i in ["", "%"]}

contracts_by_month_mod = pd.DataFrame.from_dict(new_dic)
for i in ["Cantidad", "Monto"]:
    for m in MONTH:
        contracts_by_month_mod.loc[i,(m, "%")] = (contracts_by_month_mod.loc[i,(m, "")]/contracts_by_month_mod.loc[i, (m, "")].sum() * 100).values

contracts_by_month_mod = contracts_by_month_mod.fillna(0)
contracts_by_month_mod

Ene        \
                                                                   %   
Cantidad Comparación de Precios o Competencia               32  3.80   
         Compras Menores                                   212 25.15   
         Compras por debajo del umbral mínimo              361 42.82   
         Compras por encima del umbral                       0     0   
         Licitación Privada/Restringida                      0     0   
         Licitación pública                                 19  2.25   
         Proceso de Excepción                              219 25.98   
         Sorteo de Obras                                     0     0   
Monto    Comparación de Precios o Competencia    96,910,897.80  6.69   
         Compras Menores                         61,274,289.61  4.23   
         Compras por debajo del umbral mínimo    16,094,880.54  1.11   
         Compras por encima del umbral                       0     0   
         Licitación Privada/Restringida                      0     0   
         Licitación pública                   1,220,082,502.74 84.18   
         Proceso de Excepción                    54,985,284.04  3.79   
         Sorteo de Obras                                     0     0   

                                                         Feb        \
                                                                 %   
Cantidad Comparación de Precios o Competencia             36  2.96   
         Compras Menores                                 314 25.84   
         Compras por debajo del umbral mínimo            601 49.47   
         Compras por encima del umbral                     0     0   
         Licitación Privada/Restringida                    0     0   
         Licitación pública                               20  1.65   
         Proceso de Excepción                            244 20.08   
         Sorteo de Obras                                   0     0   
Monto    Comparación de Precios o Competencia  72,218,844.97  6.11   
         Compras Menores                       99,045,709.48  8.38   
         Compras por debajo del umbral mínimo  21,411,633.24  1.81   
         Compras por encima del umbral                     0     0   
         Licitación Privada/Restringida                    0     0   
         Licitación pública                   905,401,482.77 76.65   
         Proceso de Excepción                  83,209,584.86  7.04   
         Sorteo de Obras                                   0     0   

                                                         Mar        \
                                                                 %   
Cantidad Comparación de Precios o Competencia             36  2.25   
         Compras Menores                                 386 24.08   
         Compras por debajo del umbral mínimo            864 53.90   
         Compras por encima del umbral                     0     0   
         Licitación Privada/Restringida                    0     0   
         Licitación pública                               12  0.75   
         Proceso de Excepción                            305 19.03   
         Sorteo de Obras                                   0     0   
Monto    Comparación de Precios o Competencia 127,297,700.36 23.11   
         Compras Menores                       98,679,415.31 17.91   
         Compras por debajo del umbral mínimo  30,981,374.59  5.62   
         Compras por encima del umbral                     0     0   
         Licitación Privada/Restringida                    0     0   
         Licitación pública                    77,613,396.55 14.09   
         Proceso de Excepción                 216,291,195.10 39.26   
         Sorteo de Obras                                   0     0   

                                                         Abr        \
                                                                 %   
Cantidad Comparación de Precios o Competencia             45  2.96   
         Compras Menores      

In [18]:
aux = contracts_by_month_mod.loc["Cantidad", [(m, "") for m in MONTH]]
aux = aux.transpose()
data = [go.Bar(
        x = [mod],
        y = [aux[mod].sum()],
        name = mod
        ) for mod in MOD]
iplot(go.Figure(data, layout=go.Layout(title="Cantidad de contratos aprobados por modalidad de compra")))

In [19]:
aux = contracts_by_month_mod.loc["Monto", [(m, "") for m in MONTH]]
aux = aux.transpose()
data = [go.Bar(
        x = [mod],
        y = [aux[mod].sum()],
        name = mod
        ) for mod in MOD]
iplot(go.Figure(data, layout=go.Layout(title="Montos contratados resultantes de los contratos por modalidad de compra")))

## Compras por rubro

In [20]:
num_contracts_by_rubros = pd.pivot_table(contract_sigef, values=["IdentificacionDeContrato"], index=["Rubros"], aggfunc={"IdentificacionDeContrato":np.count_nonzero})
num_contracts_by_rubros = num_contracts_by_rubros.sort_values("IdentificacionDeContrato", ascending=False)
num_contracts_by_rubros.columns = ["Cantidad"]
num_contracts_by_rubros

,Cantidad
Rubros,
Alimentos y bebidas,1115
Servicio de Mantenimiento y Reparación de Vehículos,685
Combustibles y lubricantes,626
Suministro de oficina,323
Ferreteria y pintura,313
Imprenta y publicaciones,296
"Producto medico, farmacia, laboratorio",292
"Artículos de limpieza, higiene, e insumos de cocina.",265
"Suministros, productos de tratamiento y cuidado del enfermo",252


In [21]:
aux = num_contracts_by_rubros.head(n=10)
data = [go.Bar(
        x = [rub],
        y = [aux.loc[rub, "Cantidad"]],
        name = rub
        ) for rub in aux.index]
iplot(go.Figure(data, layout=go.Layout(title="Compras por rubro")))

In [22]:
graph_pie(data=aux["Cantidad"], labels=aux.index, title="Los 10 rubro con mas contratos aprobados")

In [23]:
mount_contracts_by_rubros = pd.pivot_table(contract_sigef, values=["ValorTotal"], index=["Rubros"], aggfunc={"ValorTotal":np.sum})
mount_contracts_by_rubros = mount_contracts_by_rubros.sort_values("ValorTotal", ascending=False)
mount_contracts_by_rubros.columns = ["Monto"]
mount_contracts_by_rubros

,Monto
Rubros,
"Producto medico, farmacia, laboratorio","2,179,605,920.57"
Equipo informático y accesorios,"1,552,733,805.38"
Alimentos y bebidas,"1,088,541,295.93"
Equipo industrial para alimentos y bebidas,"382,862,481.53"
Protocolo,"330,130,706.99"
Construccion y edificacion,"258,213,652.40"
Combustibles y lubricantes,"165,573,137.40"
Ferreteria y pintura,"134,670,739.03"
Medios impresos,"129,992,196.08"


In [24]:
aux = mount_contracts_by_rubros.head(n=10)
graph_pie(data=aux["Monto"], labels=aux.index, title="Los 10 rubro con mas montos contratados")

## Compras por provincia

In [25]:
num_contracts_by_provincia = pd.pivot_table(contract_sigef, values=["IdentificacionDeContrato"], index=["Provincia"], aggfunc={"IdentificacionDeContrato":np.count_nonzero})
num_contracts_by_provincia = num_contracts_by_provincia.sort_values("IdentificacionDeContrato", ascending=False)
num_contracts_by_provincia.columns = ["Cantidad"]
num_contracts_by_provincia

,Cantidad
Provincia,
DISTRITO NACIONAL,5498
SANTO DOMINGO,1892
SANTIAGO,313
MONSENOR NOUEL,219
SAN CRISTOBAL,104
SAN JUAN,59
LA ALTAGRACIA,55
ESPAILLAT,43
LA VEGA,38


In [26]:
aux = num_contracts_by_provincia.head(n=10)
graph_pie(data=aux["Cantidad"], labels=aux.index, title="Cantidad de contratos a proveedores de esas provincias")

In [27]:
mount_contracts_by_provincia = pd.pivot_table(contract_sigef, values=["ValorTotal"], index=["Provincia"], aggfunc={"ValorTotal":np.sum})
mount_contracts_by_provincia = mount_contracts_by_provincia.sort_values("ValorTotal", ascending=False)
mount_contracts_by_provincia.columns = ["Monto"]
mount_contracts_by_provincia

,Monto
Provincia,
DISTRITO NACIONAL,"4,520,567,168.87"
SANTO DOMINGO,"2,032,956,701.48"
SANTIAGO,"994,471,786.96"
SAN CRISTOBAL,"160,387,911.20"
ESPAILLAT,"102,615,571.15"
MARIA TRINIDAD SANCHEZ,"59,397,817.76"
SAN JUAN,"38,760,303.63"
SAN JOSE DE OCOA,"37,533,330.18"
MONSENOR NOUEL,"36,504,589.81"


In [28]:
aux = mount_contracts_by_provincia.head(n=10)
graph_pie(data=aux["Monto"], labels=aux.index, title="Monto contratados a proveedores de esas provincias")

In [29]:
contracts_by_unidadCompra = pd.pivot_table(contract_sigef, values=["IdentificacionDeContrato", "ValorTotal"], index=["UnidadCompra"], aggfunc={"IdentificacionDeContrato":np.count_nonzero, "ValorTotal":np.sum})
contracts_by_unidadCompra.columns = ["Cantidad", "Monto"]
contracts_by_unidadCompra

,Cantidad,Monto
UnidadCompra,,
AUTORIDAD NACIONAL DE ASUNTOS MARITIMOS,27,"4,115,949.35"
Academia Aérea General Brigada Frank Andrés Felix Miranda,14,"1,154,114.48"
Academia Militar Batalla de las Carreras,39,"4,847,285.40"
Armada de República Dominicana,2,"634,276.40"
Astilleros Navales,12,"1,761,459.17"
Biblioteca Nacional Pedro Henríquez Ureña,89,"5,677,295.01"
CONSEJO NACIONAL DE ZONAS FRANCAS DE EXPORTACION,201,"14,139,240.04"
Centro Cardio-Neuro Oftalmologico y Trasplante,4,"1,960,737.95"
Circulo Deportivo de las FFAA y PN,48,"3,553,822.88"


## Distribucion por genero

In [30]:
GENERO = ["Femenino", "Masculino", "No Especificado"]
contracts_by_genero = pd.pivot_table(contract_sigef, values=["IdentificacionDeContrato", "ValorTotal"], index=["MesAprobacion", "GeneroUnificado"], aggfunc={"IdentificacionDeContrato":np.count_nonzero, "ValorTotal":np.sum})
contracts_by_genero = pd.DataFrame(contracts_by_genero, index=[(m, g) for m in MONTH for g in GENERO]).fillna(0)
contracts_by_genero.columns = ["Cantidad", "Monto"]

dic = contracts_by_genero.to_dict()
new_dic = {m:{(t, g):dic[t][(m,g)] for g in GENERO for t in ["Cantidad", "Monto"]} for m in MONTH}

contracts_by_genero = pd.DataFrame.from_dict(new_dic)
contracts_by_genero

Ene              Feb            Mar  \
Cantidad Femenino                     185              306            375   
         Masculino                    637              885           1207   
         No Especificado               21               24             21   
Monto    Femenino           70,927,955.10   176,272,670.09 171,608,667.07   
         Masculino       1,375,827,590.41 1,002,507,932.25 376,659,126.70   
         No Especificado     2,592,309.22     2,506,652.98   2,595,288.14   

                                      Abr              May              Jun  \
Cantidad Femenino                     403              430              318   
         Masculino                   1094             1298             1065   
         No Especificado               23               40               32   
Monto    Femenino          251,002,095.35   448,901,625.28   227,076,014.28   
         Masculino       1,089,792,401.38 1,516,803,716.15 1,250,634,645.67   
         No Especificado    53,574,469.34     4,708,806.30    19,937,106.96   

                          Jul  Ago  Sep  Oct  Nov  Dic  
Cantidad Femenino           0    0    0    0    0    0  
         Masculino          0    0    0    0    0    0  
         No Especificado    0    0    0    0    0    0  
Monto    Femenino           0    0    0    0    0    0  
         Masculino          0    0    0    0    0    0  
         No Especificado    0    0    0    0    0    0

In [31]:
aux = contracts_by_genero.loc["Cantidad"]
data = [go.Bar(
        x = aux.columns,
        y = aux.loc[g],
        name = g
        ) for g in aux.index]
iplot(go.Figure(data, layout=go.Layout(title="Cantidad de contrados mensuales por genero")))

In [32]:
aux = contracts_by_genero.loc["Cantidad"]
data  = [go.Pie(labels=aux.index, values=[aux.loc[g].sum() for g in aux.index], hole=.4)]
iplot(go.Figure(data, layout=go.Layout(title="Cantidad de contratos por genero")))

In [33]:
aux = contracts_by_genero.loc["Monto"]
data = [go.Bar(
        x = aux.columns,
        y = aux.loc[g],
        name = g
        ) for g in aux.index]
iplot(go.Figure(data, layout=go.Layout(title="Montos contratados mensuales por genero")))

In [34]:
aux = contracts_by_genero.loc["Monto"]
data  = [go.Pie(labels=aux.index, values=[aux.loc[g].sum() for g in aux.index], hole=.4)]
iplot(go.Figure(data, layout=go.Layout(title="Montos contratados por genero")))

In [37]:
DOCUMENT_TYPE = ["Cédula", "RNC", "Pasaporte", "Clave Tributaria País Extranjero"]
contracts_by_genero_femenino = contract_sigef[contract_sigef.GeneroUnificado=="Femenino"]
contracts_by_genero_femenino = pd.pivot_table(contracts_by_genero_femenino, values=["IdentificacionDeContrato", "ValorTotal"], index=["MesAprobacion", "TipoDocumento"], aggfunc={"IdentificacionDeContrato":np.count_nonzero, "ValorTotal":np.sum})
contracts_by_genero_femenino = pd.DataFrame(contracts_by_genero_femenino, index=[(m, td) for m in MONTH for td in DOCUMENT_TYPE]).fillna(0)
contracts_by_genero_femenino.columns = ["Cantidad", "Monto"]

dic = contracts_by_genero_femenino.to_dict()
new_dic = {m:{(t, td):dic[t][(m,td)] for td in DOCUMENT_TYPE for t in ["Cantidad", "Monto"]} for m in MONTH}

contracts_by_genero_femenino = pd.DataFrame.from_dict(new_dic)
contracts_by_genero_femenino

Ene            Feb  \
Cantidad Clave Tributaria País Extranjero             0              0   
         Cédula                                      12             46   
         Pasaporte                                    0              0   
         RNC                                        173            260   
Monto    Clave Tributaria País Extranjero             0              0   
         Cédula                            1,269,096.13   5,125,656.52   
         Pasaporte                                    0              0   
         RNC                              69,658,858.97 171,147,013.57   

                                                     Mar            Abr  \
Cantidad Clave Tributaria País Extranjero              0              1   
         Cédula                                       34             54   
         Pasaporte                                     0              0   
         RNC                                         341            348   
Monto    Clave Tributaria País Extranjero              0   1,937,856.08   
         Cédula                            12,146,305.15  81,961,002.34   
         Pasaporte                                     0              0   
         RNC                              159,462,361.92 167,103,236.93   

                                                     May            Jun  Jul  \
Cantidad Clave Tributaria País Extranjero              0              0    0   
         Cédula                                       45             19    0   
         Pasaporte                                     0              0    0   
         RNC                                         385            299    0   
Monto    Clave Tributaria País Extranjero              0              0    0   
         Cédula                             5,305,012.30   3,288,335.12    0   
         Pasaporte                                     0              0    0   
         RNC                              443,596,612.98 223,787,679.16    0   

                                           Ago  Sep  Oct  Nov  Dic  
Cantidad Clave Tributaria País Extranjero    0    0    0    0    0  
         Cédula                              0    0    0    0    0  
         Pasaporte                           0    0    0    0    0  
         RNC                                 0    0    0    0    0  
Monto    Clave Tributaria País Extranjero    0    0    0    0    0  
         Cédula                              0    0    0    0    0  
         Pasaporte                           0    0    0    0    0  
         RNC                                 0    0    0    0    0

## Procesos difundidos

## Compra DU-EU

## Compras vs proveedores inscritos

## Decreto 164-13

## PACC

In [44]:
currentDomain = "https://comunidad.comprasdominicana.gob.do/Public/"

header = {
    "Accept": "text/html, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "es-ES,es;q=0.9",
    "Connection": "keep-alive",
    "Content-Length": 619,
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Cookie": "PublicSessionCookie=nnlvpbtonmbzamzanz5fneu5",
    "Host": "comunidad.comprasdominicana.gob.do",
    "Origin": "https://comunidad.comprasdominicana.gob.do",
    "Referer": "https://comunidad.comprasdominicana.gob.do/Public/App/AnnualPurchasingPlanManagementPublic/Index",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

formData = {
    "startIdx": 10,
    "endIdx": 14,
    "pageNumber": 2,
    "perspective": "LastModified",
    "initAction": "Index",
    "startIndex": 6,
    "endIndex": 10,
    "currentPagingStyle": 0,
    "displayAdvancedParams": False,
    "orderParam": "VersionDateDESC",
    "searchExecuted": True
}

In [131]:
import requests
from bs4 import BeautifulSoup as bs
requests??

In [101]:
s = requests.Session()

In [128]:
posturl = "https://comunidad.comprasdominicana.gob.do/Public/App/AnnualPurchasingPlanManagementPublic/ResultListGoToPage"
r = s.get("https://comunidad.comprasdominicana.gob.do/Public/App/AnnualPurchasingPlanManagementPublic/Index")
mkey=bs(r.content, "html.parser").findAll(attrs="VortalGradualBindPaginatorButton")[0]['onclick'][96:132]

post = s.post(url=posturl+"?mkey={0}".format(mkey), data={"startidx":1, "endidx":10})

In [130]:
bs(post.content, "html.parser")

<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml">
<head id="Head"><title>
</title><meta content="Default" name="TemplateName"/>
<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/jquery-ui-1.8.2.custom.min.css_v635823209740000000" rel="stylesheet" type="text/css"/>
<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/jquery.ui.dialog.min.css_v635823209740000000" rel="stylesheet" type="text/css"/>
<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/VortalControls.min.css_v636578992150000000" rel="stylesheet" type="text/css"/>
<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/cb.min.css_v635823209720000000" rel="stylesheet" type="text/css"/>
<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/DataSheetStyles.min.css_v636410745910000000" rel="stylesheet" type="text/css"/>
<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/fullcalenda

In [99]:
r.content

b'<!DOCTYPE html>\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head id="Head"><title>\r\n\t\r\n    Annual Purchasing Plan Public\r\n\r\n</title><meta name="TemplateName" content="Public" />\n<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/jquery-ui-1.8.2.custom.min.css_v635823209740000000" rel="stylesheet" type="text/css" ></link>\n<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/jquery.ui.dialog.min.css_v635823209740000000" rel="stylesheet" type="text/css" ></link>\n<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/VortalControls.min.css_v636578992150000000" rel="stylesheet" type="text/css" ></link>\n<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/cb.min.css_v635823209720000000" rel="stylesheet" type="text/css" ></link>\n<link href="https://comunidad.comprasdominicana.gob.do/StaticContent/Styles/DataSheetStyles.min.css_v636410745910000000" rel="stylesheet" type="text/css"